In [1]:
import numpy as np
import os
import tensorflow as tf
import sys

############ Tensforflow config ############
#tf.compat.v1.disable_eager_execution()

#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.compat.v1.Session(config=config)
#tf.compat.v1.keras.backend.set_session(sess)

############## Import modules ##############
sys.path.append("../")

%load_ext autoreload
%autoreload 2

In [2]:
from modules import ucr_loader, utils
# get all datasets
root = '../../data/'
dataset_dict = ucr_loader.get_datasets(root, prefix='**/')
# retrieve data
dataset_name = 'CharacterTrajectories'
trainX, trainY, testX, testY = ucr_loader.load_data(dataset_dict[dataset_name])
# preprocess data
trainX, trainY, testX, testY = ucr_loader.preprocess_data(trainX, trainY, testX, testY, normalize=False, standardize=True)
# additional preprocessing
trainX, trainY, valX, valY = utils.perform_datasplit(trainX, trainY, test_split=0.3)
n_classes = len(np.unique(trainY))
# Shapes
print('TrainX:', trainX.shape)
print('ValX:', valX.shape)
print('TestX:', testX.shape)
print('Classes:', n_classes)

# uncomment to use the save folders
#model_dir, result_dir = utils.maybe_create_dirs(dataset_name, root='../../', dirs=['models', 'results'], return_paths=True, verbose=1)

TrainX: (995, 182, 3)
ValX: (427, 182, 3)
TestX: (1436, 182, 3)
Classes: 20


Training using a baseline model without any privacy preserving methods

In [3]:
from modules.model_definition import AlexNet
from modules import model_trainer

# uncomment to save the model
model_path = None# os.path.join(model_dir, 'AlexNet_1d_batch-' + str(options.batch_size) + '.h5')
model = AlexNet().build_1d(trainX.shape[1:], n_classes, activation='softmax', verbose=1)

model.summary()
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_trainer.train(model, trainX, trainY, validation_data=(valX, valY), epochs=100, batch_size=8, model_path=model_path, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 46, 96)            3264      
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 96)            384       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 23, 96)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 23, 256)           123136    
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 256)           1024      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 12, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 12, 384)           2

In [4]:
# uncomment to save the model
report_path = None#os.path.join(result_dir, 'AlexNet_1d_batch-' + str(options.batch_size) + '_report.txt')
preds = np.argmax(model.predict(testX, batch_size=32, verbose=1), axis=1)
utils.compute_classification_report(testY, preds, save=report_path, verbose=1, store_dict=True)

45/45 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        86
           1     0.9538    0.9538    0.9538        65
           2     1.0000    0.9859    0.9929        71
           3     1.0000    1.0000    1.0000        66
           4     1.0000    1.0000    1.0000        62
           5     1.0000    0.9833    0.9916        60
           6     1.0000    1.0000    1.0000        67
           7     0.9851    1.0000    0.9925        66
           8     1.0000    1.0000    1.0000        78
           9     0.9836    0.9524    0.9677        63
          10     1.0000    1.0000    1.0000        69
          11     1.0000    1.0000    1.0000        71
          12     1.0000    1.0000    1.0000        86
          13     0.9861    1.0000    0.9930        71
          14     1.0000    1.0000    1.0000        79
          15     0.9894    1.0000    0.9947        93
          16     1.0000    1